In [2]:
%%HTML
<p id = 'heading1_p'>Secure Boot - PIC32CZCA90/PIC32CKSG01 </p>
<hr id = 'hline'> </hr>
<p id = 'heading1_p' style='font-size: 20px; font-weight: bold;'>Introduction</p>
<div id='div_element'>
<p>This usecase will sign and authenticate application image to support secure boot in combination with HSM embedded in PIC32CZ-CA90/PIC32CK-SG01.
The use case demonstrates Sequential Secure Boot of PIC32CZ-CA90/PIC32CK-SG01 in Test Mode.  The HSM authenticates meta data and then verifies HSM and host application image before execution. If there is an authentication failure the secure boot will not transfer control to host application and the board could be permanently locked out. To avoid this and safely evaluate secure boot, this use case uses Test Secure Boot Mode to help with secure boot development before production.
</p>
</div>

<button class= "collapsible" style='font-size: 20px; font-weight: bold;'>   Cryptographic Asset list & Use Case implementation</button>
<div class="content">
<div id='content_element'>
<hr id = 'h_content'> </hr>
<p style="float: left; font-size: 9pt; text-align: center; 20em; width: 55%; margin-right: 1%;"><img src="assets\images\secureboot_steps.png" style="height: 60em; width: 100%"><b>Use Case implementation</b></p>
<p style="clear: both;">
</div>
</div>

<div id='div_element'>
<p>Select your Evaluation kit and then click on each step to execute the Use Case. Click on Usecase Help button in transaction diagram for additional details.</p>
</div>
<div id='div_element'>
<p><h3>Note:</h3> In step 2 select led_app_pic32czca90 for PIC32CZCA90 CUltra and led_app_pic32cksg01 for PIC32CKSG01 CUltra.Use prodiction hex image for custom application. </p>
</div>

In [1]:
import os, shutil
from tpds.usecase_diagram import ProtoBoard, UsecaseDiagram
from tpds.tp_utils.tp_settings import TPSettings
from tpds.proto_boards import get_board_path
from IPython.display import display

curr_path = os.path.abspath('')
os.sys.path.extend([os.path.join(os.getcwd(), 'assets', 'python')])
import secureboot

tp_settings = TPSettings()
usecase_working_dir = os.path.join(tp_settings.get_base_folder(), 'HSM_secureboot')
os.makedirs(usecase_working_dir, exist_ok=True)
boards = ProtoBoard(max_width=400)



app_path_pic32czca90=os.path.join(curr_path,'..','backend','boards','PIC32CZCA90', 'led_app_pic32czca90.hex')
fwmdtool_path_pic32czca90=os.path.join(curr_path,'..','backend','Tools')
hsm_hex_pic32czca90=os.path.join(curr_path,'..','backend','boards','PIC32CZCA90', 'HSM_pic32czca90.hex')
shutil.copy(app_path_pic32czca90, usecase_working_dir)
shutil.copy(hsm_hex_pic32czca90, usecase_working_dir)


app_path_pic32cksg01=os.path.join(curr_path,'..','backend','boards','PIC32CKSG01', 'led_app_pic32cksg01.hex')
hex_path_pic32cksg01=os.path.join(curr_path,'..','backend','boards','PIC32CKSG01', 'HSM_pic32cksg01.hex')
shutil.copy(hex_path_pic32cksg01, usecase_working_dir)
shutil.copy(app_path_pic32cksg01, usecase_working_dir)


fwmdtool_path=os.path.join(curr_path,'..','backend','Tools')
destination=shutil.copytree(fwmdtool_path, usecase_working_dir,dirs_exist_ok=True)


boards.add_board(
            'PIC32CZCA90 CUltra',
            os.path.join(curr_path,'..','backend','boards','PIC32CZCA90','PIC32CZCA90.png'),
            os.path.join(curr_path,'..','backend','boards','PIC32CZCA90', 'kit_image_pic32czca90.hex'),
            os.path.join(curr_path,'firmware','pic32czca_led_app','pic32cz_ca90_LEDApp.X')
            )
boards.add_board(
            'PIC32CKSG01 CUltra',
            os.path.join(curr_path,'..','backend','boards','PIC32CKSG01','PIC32CKSG01.png'),
            os.path.join(curr_path,'..','backend','boards','PIC32CKSG01', 'kit_image_pic32cksg01.hex'),
            os.path.join(curr_path,'firmware','pic32cksg_led_app', 'led_app', 'led_app_pic32cksg.X')
            )
boards.render_boards(0)

boards.render_boards(0)



sec_boot = secureboot.SecureBootUsecase(boards)
usecase_td = UsecaseDiagram(os.path.join(curr_path,'assets','images','secureboot_td.png'), working_dir=usecase_working_dir)

#1
kit_protocol_hex = usecase_td.add_script(235,62,30,30, sec_boot.generate_resources)

#2
app_hex = usecase_td.add_script(235,102,30,30, sec_boot.app_hex,prereq_scripts = [kit_protocol_hex])

#3
xml_modify = usecase_td.add_script(235,149,30,30, sec_boot.flash_xml_modify,prereq_scripts = [app_hex])

#4
combined_hex= usecase_td.add_script(235,197,30,30,sec_boot.FWMDtool_combined_hex,prereq_scripts = [xml_modify])

#5
flash_output_hex= usecase_td.add_script(235,336,30,30,sec_boot.prog_FWMD,prereq_scripts = [combined_hex])

usecase_td.add_firmware(boards.get_firmware_project, tp_settings.get_mplab_paths().get('ide_path'))
usecase_td.add_image(726,262,141,27, os.path.join(curr_path,'assets','images', 'c_snippet_template.PNG'), 'Code Snippet')
usecase_td.add_usecase_help(os.path.join(curr_path,'assets', 'HSM-SecureBoot.md'))

display(boards.canvas)
usecase_td.display_canvas()

MultiCanvas(height=200, layout=Layout(margin='50px'), width=400)